In [1]:
import os
import re
import random

import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn import model_selection
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import scipy.linalg

(1) Pick k = 5 classes of your choice from the dataset. You may choose any subset of 5 classes among all classes of the dataset.

In [ ]:
def list_files(dir):
    folders = []
    for root, dirs, files in os.walk(dir):
        for r in dirs:
            folders.append(r)
    return folders

def list_directories(folders_list):
    images_dirs = []
    for i in range(len(folders)):
        images_d = '/Users/jingyany/Desktop/17Spring/Data558/data competation/five classes/'+folders[i]+'/'
        images_dirs.append(images_d)
    return images_dirs

def images_lists(images_dirs):
    list_images = []
    for i in range(len(images_dirs)):
        images_dir = images_dirs[i]
        for f in os.listdir(images_dir):
            if re.search('jpg|JPG', f):
                list_images.append(images_dir + f)
    return list_images

In [ ]:
model_dir = '/Users/jingyany/models/tutorials/image/imagenet/TUTORIAL_DIR/imagenet'
dir = '/Users/jingyany/Desktop/17Spring/Data558/data competation/five classes'
folders = list_files(dir)
images_dirs = list_directories(folders)
list_images = images_lists(images_dirs)

In [ ]:
def create_graph():
    with gfile.FastGFile(os.path.join(model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
        


In [ ]:
def extract_features(list_images):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    labels = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

    for ind, image in enumerate(list_images):
        #if (ind%100 == 0):
        print('Processing %s...' % (image))
        if not gfile.Exists(image):
            tf.logging.fatal('File does not exist %s', image)

        image_data = gfile.FastGFile(image, 'rb').read()
        predictions = sess.run(next_to_last_tensor,{'DecodeJpeg/contents:0': image_data})
        features[ind,:] = np.squeeze(predictions)
        labels.append(re.split('_\d+',image.split('/')[-2].split('.')[0])[0])
    labels = list(map(int, labels))

    return features, labels

In [ ]:
#features,labels = extract_features(list_images)

(2) Write a function that, for any class at hand, creates a training set with an equal number of examples from the class at hand and from the other classes. You may simply randomly pick the examples from the other classes.

In [2]:
features = pickle.load(open('features_5class','rb'))
labels = pickle.load(open('labels_5class','rb'))

In [3]:
def create_training(features, labels, start, num=30):
    if start == 0:
        X_one_class = features[:num]
        y_one_class = labels[:num]
        X_other_classes = features[num:]
        y_other_classes = labels[num:]
    else: 
        X_one_class = features[start:(start+num)]
        y_one_class = labels[start:(start+num)]
        X_other_classes = np.concatenate([features[:start], features[(start+num):]])
        y_other_classes = np.concatenate([labels[:start], labels[(start+num):]])
    index = np.random.choice(y_other_classes.shape[0],num,False)
    index = index.tolist()
    y_other_classes_num= np.zeros(num)
    X_other_classes_num = np.zeros((num,2048))
    
    for i in range(len(index)):
        X_other_classes_num[i] = X_other_classes[index[i]]
        y_other_classes_num[i] = y_other_classes[index[i]]
        
    return X_one_class, y_one_class, X_other_classes_num, y_other_classes_num

(3) For each class c, train an $l_2^2$-regularized logistic regression classifier using your own fast gradient algorithm with $\lambda_c$ = 1. Display the confusion matrix. Which classes seem to be the most difficult to classify?

In [26]:
def computegrad(beta, lamda, x, y):
    n = len(y)
    yx = y[:, None]*x
    upper = yx*np.exp(-yx.dot(beta[:, None]))
    bottom = np.exp(-yx.dot(beta))+1
    gradient = -1/n *np.sum(upper/bottom[:, None], axis=0) + 2*lamda*beta
                            
    return gradient

def function(beta, lamda, x, y):
    n = len(y)
    yx = y[:, None]*x
    f = 1/n*(np.sum(np.log(np.exp(-yx.dot(beta))+1))) + lamda*np.linalg.norm(beta)**2
    return f

def backtracking(beta, lamda, t=1, alpha=0.5, beta_s=0.8, max_iter=100):
    grad_beta = computegrad(beta, lamda, x=X_train, y=y_train)
    norm_grad_beta = np.linalg.norm(grad_beta)
    found_t = 0 
    iter = 0 
    while (found_t == 0 and iter < max_iter):       
        if (function(beta - t*grad_beta, lamda, x=X_train, y=y_train)) < (function(beta, lamda, x=X_train, y=y_train)-alpha*t*(norm_grad_beta)**2):
            found_t = 1        
        elif(iter == max_iter):
            stop("Maximum number of iterations reached")         
        else:
            t = t*beta_s
            iter = iter + 1   
    return t 

def fastgradalgo(beta_init,theta,lamda,t_init, max_iter=500):
    beta = beta_init
    grad_theta = computegrad(beta, lamda, x=X_train, y=y_train)
    beta_vals = theta
    
    iter = 0
    while(iter < max_iter):         
        t = backtracking(beta, lamda)
        beta1 = theta - t*grad_theta
        theta = beta1 + t/(t+3)*(beta1 - beta)
        beta_vals = np.vstack((beta_vals, theta))
        grad_theta = computegrad(theta, lamda, x=X_train, y=y_train)
        beta = beta1
        iter = iter + 1
        
    return beta_vals

def get_predicted(beta_opt, x):
    y_pred = 1/(1+np.exp(-x.dot(beta_opt))) > 0.5
    y_pred = y_pred*2 - 1  # Convert to +/- 1
    return y_pred

In [18]:
def redefine_lables(labels, number):
    for i in range(labels.shape[0]):
        if labels[i] == number:
            labels[i] = 1
        else:
            labels[i] = -1
    return labels

In [22]:
def training_test(X, y, X_rest, y_rest, number, first_class):
    X_new = np.concatenate([X,X_rest])
    y_new = np.concatenate([y,y_rest])
    y_new = redefine_lables(y_new, first_class)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

In [27]:
def get_pred_test(X_train, X_test, y_train, y_test):
    d = np.size(X_train, 1)
    beta = np.zeros(d)
    theta = np.zeros(d)
    lambduh = 1
    eta_init = 1/(scipy.linalg.eigh(1/len(y_train)*X_train.T.dot(X_train), eigvals=(d-1, d-1), eigvals_only=True)[0]+lambduh)
    fgd_betas = fastgradalgo(beta_init=beta,theta=theta,lamda=1,t_init=eta_init)
    y_pred = get_predicted(fgd_betas[-1], X_test)
    return y_pred 

#metrics.confusion_matrix(y_test1, y_pred)

In [20]:
#Use class 1 versus rest classes
X1,y1,X_rest_1, y_rest_1 = create_training(features, labels, 0)
y1

array([44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
       44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44])

In [23]:
X_train1, X_test1, y_train1, y_test1 = training_test(X1,y1,X_rest_1, y_rest_1, 0, 44)

In [29]:
X_train = X_train1
y_train = y_train1
y_pred1 = get_pred_test(X_train1, X_test1, y_train1, y_test1)

In [30]:
print(y_pred1)
print(y_test1)

[ 1  1  1 -1  1 -1 -1 -1  1 -1 -1 -1]
[ 1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.]


In [31]:
class1_cm = metrics.confusion_matrix(y_test1, y_pred1)
class1_cm

array([[7, 1],
       [0, 4]])

In [32]:
#Use class 2 versus rest classes
X2,y2,X_rest_2, y_rest_2 = create_training(features, labels, 30)
y2

array([45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45,
       45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45])

In [33]:
X_train2, X_test2, y_train2, y_test2 = training_test(X2,y2,X_rest_2, y_rest_2, 30, 45)

In [34]:
X_train = X_train2
y_train = y_train2
y_pred2 = get_pred_test(X_train2, X_test2, y_train2, y_test2)

In [35]:
print(y_pred2)
print(y_test2)

[ 1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1]
[ 1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.]


In [36]:
class2_cm = metrics.confusion_matrix(y_test2, y_pred2)
class2_cm

array([[8, 0],
       [1, 3]])

In [37]:
#Use class 3 versus rest classes
X3,y3,X_rest_3, y_rest_3 = create_training(features, labels, 60)
y3

array([46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46,
       46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46])

In [38]:
X_train3, X_test3, y_train3, y_test3 = training_test(X3,y3,X_rest_3, y_rest_3, 60, 46)

In [39]:
X_train = X_train3
y_train = y_train3
y_pred3 = get_pred_test(X_train3, X_test3, y_train3, y_test3)

In [40]:
print(y_pred3)
print(y_test3)

[ 1  1 -1 -1  1 -1 -1 -1  1 -1 -1 -1]
[ 1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.]


In [41]:
class3_cm = metrics.confusion_matrix(y_test3, y_pred3)
class3_cm

array([[8, 0],
       [0, 4]])

In [42]:
#Use class 4 versus rest classes
X4,y4,X_rest_4, y_rest_4 = create_training(features, labels, 90)
y4

array([47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47,
       47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47])

In [43]:
X_train4, X_test4, y_train4, y_test4 = training_test(X4,y4,X_rest_4, y_rest_4, 90, 47)

In [44]:
X_train = X_train4
y_train = y_train4
y_pred4 = get_pred_test(X_train4, X_test4, y_train4, y_test4)

In [45]:
print(y_pred4)
print(y_test4)

[ 1  1 -1 -1  1 -1 -1 -1  1 -1 -1 -1]
[ 1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.]


In [46]:
class4_cm = metrics.confusion_matrix(y_test4, y_pred4)
class4_cm

array([[8, 0],
       [0, 4]])

In [47]:
#Use class 5 versus rest classes
X5,y5,X_rest_5, y_rest_5 = create_training(features, labels, 120)
y5

array([48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48,
       48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48])

In [48]:
X_train5, X_test5, y_train5, y_test5 = training_test(X5,y5,X_rest_5, y_rest_5, 120, 48)

In [49]:
X_train = X_train5
y_train = y_train5
y_pred5 = get_pred_test(X_train5, X_test5, y_train5, y_test5)

In [50]:
print(y_pred5)
print(y_test5)

[ 1  1  1  1  1 -1 -1 -1  1  1 -1  1]
[ 1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.]


In [51]:
class5_cm = metrics.confusion_matrix(y_test5, y_pred5)
class5_cm

array([[4, 4],
       [0, 4]])

In [52]:
print(class1_cm)
print(class2_cm)
print(class3_cm)
print(class4_cm)
print(class5_cm)

[[7 1]
 [0 4]]
[[8 0]
 [1 3]]
[[8 0]
 [0 4]]
[[8 0]
 [0 4]]
[[4 4]
 [0 4]]


Anwser: class 5 seems to be the most difficult to classify, since the number of correctly predicted lables is the smallest among all 5 classes.

(4) Write a function that returns the ranked list of classes in terms of classification difficulty using the confusion matrix. Compute the multi-class misclassification error.

In [53]:
cm_list = [class1_cm, class2_cm, class3_cm, class4_cm, class5_cm]

In [54]:
def ranked_list(matrix_list):
    correct_list = []
    ranked_list = []
    for i in range(5):
        correct = cm_list[i][0][0]+cm_list[i][1][1]
        correct_list.append(correct)
    for j in range(5):
        class_num = np.argmin(correct_list)+1
        ranked_list.append(class_num)
        correct_list[np.argmin(correct_list)] = 13
    return ranked_list

In [55]:
ranked_list(cm_list)

[5, 1, 2, 3, 4]

In [56]:
incorrect_list = []
for i in range(5):
    incorrect = cm_list[i][0][1]+cm_list[i][1][0]
    incorrect_list.append(incorrect)
print("The multi-class misclassification error is", + sum(incorrect_list)/60)

The multi-class misclassification error is 0.1


Answer: the multi-class misclassification error of those 5 classes is about 0.1.

(5) Find the values of the regularization parameters $\lambda_1,..., \lambda_k$ for the classifiers using a hold-out validation set strategy. Define a grid of values $\Lambda$ for each parameter $\lambda_c$ with c = 1,..., k. For each setting of the regularization parameters $\lambda_1,..., \lambda_k$, where each $\lambda_c$ can take values in $\Lambda$ (independently), train all your k = 5 classifiers and save the multi-class misclassification error on the validation set for each setting of the regularization parameters $\lambda_1,..., \lambda_k$

In [57]:
def compute_misclassification_error(beta_opt, x, y):
    y_pred = 1/(1+np.exp(-x.dot(beta_opt))) > 0.5
    y_pred = y_pred*2 - 1 
    return np.mean(y_pred != y)

In [59]:
#misclassification error for classifier 1
lambdas = [10.0**i for i in np.arange(-3, 3, 1)]
misclsf_error_1 = np.zeros_like(lambdas)
for i in range(len(lambdas)):
    lambduh = lambdas[i]
    d = np.size(X_train1, 1)
    beta_init = np.zeros(d)
    X_train = X_train1
    y_train = y_train1
    X_test = X_test1
    y_test = y_test1
    eta_init = 1/(scipy.linalg.eigh(1/len(y_train)*X_train.T.dot(X_train), eigvals=(d-1, d-1), eigvals_only=True)[0]+lambduh)
    betas_rand = fastgradalgo(beta_init=beta_init,theta=beta_init,lamda=lambduh,t_init=eta_init)
    misclsf_error_1[i] = compute_misclassification_error(betas_rand[-1], X_test, y_test)
print('Smallest misclassification error value:', min(misclsf_error_1), 'at lambda =', lambdas[np.argmin(misclsf_error_1)])

Smallest misclassification error value: 0.0 at lambda = 0.001


In [60]:
#misclassification error for classifier 2
lambdas = [10.0**i for i in np.arange(-3, 3, 1)]
misclsf_error_2 = np.zeros_like(lambdas)
for i in range(len(lambdas)):
    lambduh = lambdas[i]
    d = np.size(X_train2, 1)
    beta_init = np.zeros(d)
    X_train = X_train2
    y_train = y_train2
    X_test = X_test2
    y_test = y_test2
    eta_init = 1/(scipy.linalg.eigh(1/len(y_train)*X_train.T.dot(X_train), eigvals=(d-1, d-1), eigvals_only=True)[0]+lambduh)
    betas_rand = fastgradalgo(beta_init=beta_init,theta=beta_init,lamda=lambduh,t_init=eta_init)
    misclsf_error_1[i] = compute_misclassification_error(betas_rand[-1], X_test, y_test)
print('Smallest misclassification error value:', min(misclsf_error_2), 'at lambda =', lambdas[np.argmin(misclsf_error_2)])

Smallest misclassification error value: 0.0 at lambda = 0.001


In [61]:
#misclassification error for classifier 3
lambdas = [10.0**i for i in np.arange(-3, 3, 1)]
misclsf_error_3 = np.zeros_like(lambdas)
for i in range(len(lambdas)):
    lambduh = lambdas[i]
    d = np.size(X_train3, 1)
    beta_init = np.zeros(d)
    X_train = X_train3
    y_train = y_train3
    X_test = X_test3
    y_test = y_test3
    eta_init = 1/(scipy.linalg.eigh(1/len(y_train)*X_train.T.dot(X_train), eigvals=(d-1, d-1), eigvals_only=True)[0]+lambduh)
    betas_rand = fastgradalgo(beta_init=beta_init,theta=beta_init,lamda=lambduh,t_init=eta_init)
    misclsf_error_1[i] = compute_misclassification_error(betas_rand[-1], X_test, y_test)
print('Smallest misclassification error value:', min(misclsf_error_3), 'at lambda =', lambdas[np.argmin(misclsf_error_3)])

Smallest misclassification error value: 0.0 at lambda = 0.001


In [62]:
#misclassification error for classifier 4
lambdas = [10.0**i for i in np.arange(-3, 3, 1)]
misclsf_error_4 = np.zeros_like(lambdas)
for i in range(len(lambdas)):
    lambduh = lambdas[i]
    d = np.size(X_train4, 1)
    beta_init = np.zeros(d)
    X_train = X_train4
    y_train = y_train4
    X_test = X_test4
    y_test = y_test4
    eta_init = 1/(scipy.linalg.eigh(1/len(y_train)*X_train.T.dot(X_train), eigvals=(d-1, d-1), eigvals_only=True)[0]+lambduh)
    betas_rand = fastgradalgo(beta_init=beta_init,theta=beta_init,lamda=lambduh,t_init=eta_init)
    misclsf_error_1[i] = compute_misclassification_error(betas_rand[-1], X_test, y_test)
print('Smallest misclassification error value:', min(misclsf_error_4), 'at lambda =', lambdas[np.argmin(misclsf_error_4)])

Smallest misclassification error value: 0.0 at lambda = 0.001


In [63]:
#misclassification error for classifier 5
lambdas = [10.0**i for i in np.arange(-3, 3, 1)]
misclsf_error_5 = np.zeros_like(lambdas)
for i in range(len(lambdas)):
    lambduh = lambdas[i]
    d = np.size(X_train5, 1)
    beta_init = np.zeros(d)
    X_train = X_train5
    y_train = y_train5
    X_test = X_test5
    y_test = y_test5
    eta_init = 1/(scipy.linalg.eigh(1/len(y_train)*X_train.T.dot(X_train), eigvals=(d-1, d-1), eigvals_only=True)[0]+lambduh)
    betas_rand = fastgradalgo(beta_init=beta_init,theta=beta_init,lamda=lambduh,t_init=eta_init)
    misclsf_error_1[i] = compute_misclassification_error(betas_rand[-1], X_test, y_test)
print('Smallest misclassification error value:', min(misclsf_error_5), 'at lambda =', lambdas[np.argmin(misclsf_error_5)])

Smallest misclassification error value: 0.0 at lambda = 0.001


(6) Find the optimal value of the regularization parameters $\lambda_1,..., \lambda_k$ based on the validation error. Display the confusion matrix for this setting of the regularization param-
eters.

In [64]:
#The optimal lambda is 0.001 for all five classifiers
def get_pred_test_opt(X_train, X_test, y_train, y_test):
    d = np.size(X_train, 1)
    beta = np.zeros(d)
    theta = np.zeros(d)
    lambduh = 0.001
    eta_init = 1/(scipy.linalg.eigh(1/len(y_train)*X_train.T.dot(X_train), eigvals=(d-1, d-1), eigvals_only=True)[0]+lambduh)
    fgd_betas = fastgradalgo(beta_init=beta,theta=theta,lamda=1,t_init=eta_init)
    y_pred = get_predicted(fgd_betas[-1], X_test)
    return y_pred 

In [65]:
X_train = X_train1
y_train = y_train1
y_pred1_opt = get_pred_test_opt(X_train1, X_test1, y_train1, y_test1)
class1_cm_opt = metrics.confusion_matrix(y_test1, y_pred1_opt)

In [66]:
X_train = X_train2
y_train = y_train2
y_pred2_opt = get_pred_test_opt(X_train2, X_test2, y_train2, y_test2)
class2_cm_opt = metrics.confusion_matrix(y_test2, y_pred2_opt)

In [67]:
X_train = X_train3
y_train = y_train3
y_pred3_opt = get_pred_test_opt(X_train3, X_test3, y_train3, y_test3)
class3_cm_opt = metrics.confusion_matrix(y_test3, y_pred3_opt)

In [68]:
X_train = X_train4
y_train = y_train4
y_pred4_opt = get_pred_test_opt(X_train4, X_test4, y_train4, y_test4)
class4_cm_opt = metrics.confusion_matrix(y_test4, y_pred4_opt)

In [69]:
X_train = X_train5
y_train = y_train5
y_pred5_opt = get_pred_test_opt(X_train5, X_test5, y_train5, y_test5)
class5_cm_opt = metrics.confusion_matrix(y_test5, y_pred5_opt)

In [70]:
print(class1_cm_opt)
print(class2_cm_opt)
print(class3_cm_opt)
print(class4_cm_opt)
print(class5_cm_opt)

[[7 1]
 [0 4]]
[[8 0]
 [1 3]]
[[8 0]
 [0 4]]
[[8 0]
 [0 4]]
[[4 4]
 [0 4]]
